# Базовые модели

- Naive baseline: завтра = сегодня
- EWMA baseline: скользящий стандарт

Используется для контроля адекватности продвинутых моделей.


In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

print("✅ Библиотеки загружены")


## Базовые модели прогноза волатильности


In [ ]:
def naive_forecast(returns, window=30):
    """
    Naive baseline: прогноз волатильности = текущая волатильность
    """
    current_vol = returns.rolling(window=window).std() * np.sqrt(252)
    forecast = current_vol.shift(1)  # Прогноз = предыдущее значение
    return forecast


def ewma_forecast(returns, span=30):
    """
    EWMA baseline: экспоненциально взвешенная волатильность
    """
    ewma_vol = returns.ewm(span=span).std() * np.sqrt(252)
    forecast = ewma_vol.shift(1)
    return forecast


def calculate_mae(actual, predicted):
    """Mean Absolute Error"""
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    return np.mean(np.abs(actual[mask] - predicted[mask]))


def calculate_rmse(actual, predicted):
    """Root Mean Squared Error"""
    mask = ~(np.isnan(actual) | np.isnan(predicted))
    return np.sqrt(np.mean((actual[mask] - predicted[mask])**2))

print("✅ Функции базовых моделей загружены")


## Загрузка данных и создание прогнозов


In [ ]:
# Загрузка данных с целевыми переменными
DATA_DIR = Path('data') / 'features'
ticker = 'SBER'
df = pd.read_parquet(DATA_DIR / f"{ticker}_with_targets.parquet")

# Создание прогнозов
df['forecast_naive'] = naive_forecast(df['log_return'], window=30)
df['forecast_ewma'] = ewma_forecast(df['log_return'], span=30)

# Оценка качества на целевой переменной (5-дневная волатильность)
target_col = 'target_vol_5d'

mae_naive = calculate_mae(df[target_col].values, df['forecast_naive'].values)
mae_ewma = calculate_mae(df[target_col].values, df['forecast_ewma'].values)

rmse_naive = calculate_rmse(df[target_col].values, df['forecast_naive'].values)
rmse_ewma = calculate_rmse(df[target_col].values, df['forecast_ewma'].values)

print(f"\n=== Метрики базовых моделей ({ticker}) ===")
print(f"\nNaive Baseline:")
print(f"  MAE:  {mae_naive:.4f}")
print(f"  RMSE: {rmse_naive:.4f}")

print(f"\nEWMA Baseline:")
print(f"  MAE:  {mae_ewma:.4f}")
print(f"  RMSE: {rmse_ewma:.4f}")

print(f"\n📊 Пример прогнозов:")
print(df[['date', target_col, 'forecast_naive', 'forecast_ewma']].tail(10))


## Сохранение результатов


In [ ]:
OUTPUT_DIR = Path('data') / 'models'
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Сохранение результатов
output_path = OUTPUT_DIR / f"{ticker}_baseline_forecasts.parquet"
df.to_parquet(output_path, index=False)

# Сохранение метрик
metrics = pd.DataFrame({
    'model': ['Naive', 'EWMA'],
    'mae': [mae_naive, mae_ewma],
    'rmse': [rmse_naive, rmse_ewma]
})
metrics_path = OUTPUT_DIR / f"{ticker}_baseline_metrics.csv"
metrics.to_csv(metrics_path, index=False)

print(f"✅ Сохранено: {output_path}")
print(f"✅ Метрики: {metrics_path}")
